In [1]:

%pwd

'e:\\Projects\\Text-Summarization-Project\\research'

In [2]:
import os
os.chdir("../")

In [3]:
%pwd

'e:\\Projects\\Text-Summarization-Project'

In [4]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir : Path
    data_path : Path
    model_pkg : Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    per_device_eval_batch_size : int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

In [5]:
from textsummarizer.constants import *
from textsummarizer.utils.common import read_yaml , create_directories

In [6]:
class ConfigurationManager:
    def __init__ (
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params =self.params.TrainingArguments

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path = config.data_path,
            model_pkg=  config.model_pkg,
            num_train_epochs= params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size= params.per_device_train_batch_size,
            per_device_eval_batch_size=params.per_device_eval_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
            

        )

        return model_trainer_config

In [7]:
from transformers import AutoModelForSeq2SeqLM , AutoTokenizer
from transformers import TrainingArguments , Trainer
from transformers import DataCollatorForSeq2Seq
from datasets import load_dataset , load_from_disk
import torch

c:\Users\LENOVO\anaconda3\envs\tsum\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-04-22 23:31:01,569: INFO: config: PyTorch version 2.2.2 available.]


In [8]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer =  AutoTokenizer.from_pretrained(self.config.model_pkg)
        model_pegas  = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_pkg).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegas)


        #loading dataset
        dataset = load_from_disk(self.config.data_path)

        # trainer_args = TrainingArguments(
        #     output_dir= self.config.root_dir, 
        #     num_train_epochs=self.config.num_train_epochs,
        #     warmup_steps=self.config.warmup_steps,
        #     per_device_train_batch_size=self.config.per_device_train_batch_size, 
        #     per_device_eval_batch_size=self.config.per_device_eval_batch_size,
        #     weight_decay= self.config.weight_decay, 
        #     logging_steps=self.config.logging_steps,
        #     evaluation_strategy=self.config.evaluation_strategy, 
        #     eval_steps=self.config.eval_steps, 
        #     save_steps=self.config.save_steps,
        #     gradient_accumulation_steps=self.config.gradient_accumulation_steps
        #     )
        
        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
            per_device_train_batch_size=1, per_device_eval_batch_size=1,
            weight_decay=0.01, logging_steps=10,
            evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
            gradient_accumulation_steps=16
        )
        
        trainer = Trainer(model=model_pegas, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset["test"],
                  eval_dataset=dataset["validation"])
        
        trainer.train()

        #save model
        model_pegas.save_pretrained(os.path.join(self.config.root_dir,"Text-Summarizer-model"))


        # Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))
        

In [9]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config= model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2024-04-22 23:31:05,725: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-22 23:31:05,729: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-22 23:31:05,729: INFO: common: created directory at: artifacts]
[2024-04-22 23:31:05,730: INFO: common: created directory at: artifacts/model_trainer]


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
 20%|█▉        | 10/51 [06:14<26:30, 38.79s/it]

{'loss': 3.1101, 'grad_norm': 31.934059143066406, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.2}


 39%|███▉      | 20/51 [13:16<23:10, 44.86s/it]

{'loss': 3.0467, 'grad_norm': 15.838595390319824, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.39}


 59%|█████▉    | 30/51 [23:18<21:25, 61.21s/it]

{'loss': 3.1613, 'grad_norm': 10.26186752319336, 'learning_rate': 3e-06, 'epoch': 0.59}


 75%|███████▍  | 38/51 [29:54<08:38, 39.85s/it]